## CAPSTONE PROJECT
### Case: *Recommendation System*
### Judul: Perancangan Sistem Pemberian Rekomendasi Buku Website X Menggunakan Metode *Collaborative Based Filtering*
### Disusun Oleh: Fikri Irfan Adristi
---



#### IMPORT PACKAGES

In [107]:
# Load library/package
import os
import pandas as pd
import numpy as np

# Scipy
from scipy.sparse import csr_matrix

# Sklearn
from sklearn.neighbors import NearestNeighbors

#### READ DATASET

In [108]:
# Connect Colab to GDrive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [109]:
# Mendefinisikan path sesuai folder GDrive dimana file dataset penyusun project disimpan
path ='/content/drive/MyDrive/Narasio/Certified Data Scientist/Senior Data Scientist/Capstone Project/Dataset/'

In [110]:
# Cek isi direktori file
os.listdir(path)

['Books.csv', 'Ratings.csv', 'Users.csv']

Terdapat tiga dataset dalam direktori file yaitu: Books.csv, Ratings.csv, dan Users.csv.

In [111]:
# Membaca dataset
df_books = pd.read_csv(path + 'Books.csv')
df_users = pd.read_csv(path + 'Users.csv')
df_ratings = pd.read_csv(path + 'Ratings.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [112]:
# Display lima books data teratas
df_books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [113]:
# Display lima users data teratas
df_users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [114]:
# Display lima ratings data teratas
df_ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


#### PROBLEM STATEMENT

Sistem pemberi rekomendasi adalah sebuah sistem yang menggunakan algoritma untuk menyarankan item relevan seperti film, produk elektronik, hingga buku. Sebuah website X untuk penggemar baca buku juga menerapkan sistem rekomendasi sehingga, *problem statement* yang tepat untuk dalam kasus website X ini adalah:

Bagaimana perancangan sistem pemberi rekomendasi buku yang tepat kepada penggemar berdasarkan *rating* buku yang sering dibaca dan variabel lainnya?

#### DATA PRE-PROCESSING

Memunculkan Informasi Tentang Jumlah dan Tipe Data dari Masing-masing Dataset

In [115]:
# Memunculkan informasi jumlah data dan tipe data tiap kolom dataset books.
df_books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271359 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


*   Dari informasi diatas diketahui bahwa terdapat perbedaan jumlah data pada 
masing masing kolom. Pada *"Range Index"* tertulis 271360 data terisi, sedangkan kolom Book-Author, Publisher, dan Image-URL-L hanya memiliki jumlah data yang kurang dari 271360. Hal tersebut mengindikasikan bahwa terdapat *missing value* pada kolom Book-Author, Publisher, dan Image-URL-L.
*   Kolom Image-URL-S, Image-URL-M, dan Image-URL-L akan dihilangkan *(drop)* karena tidak dibutuhkan untuk analisis lebih lanjut.



In [116]:
# Memunculkan informasi jumlah data dan tipe data tiap kolom dataset users.
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


Dari informasi diatas diketahui bahwa terdapat perbedaan jumlah data pada 
masing masing kolom. Pada "*Range Index*" tertulis 278858 data terisi, sedangkan kolom Age hanya memiliki jumlah data yang kurang dari 278858. Hal tersebut mengindikasikan bahwa terdapat *missing value* pada kolom Age.

In [117]:
# Memunculkan informasi jumlah data dan tipe data tiap kolom dataset users.
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


Dari informasi diatas diketahui bahwa terdapat perbedaan jumlah data pada 
masing masing kolom. Pada "*Range Index*" tertulis 526356 data terisi, dan semua kolom memiliki jumlah data yang sama. Hal tersebut mengindikasikan bahwa tidak terdapat *missing value* pada kolom seluruh.

Menghilangkan *(Drop)* Kolom yang Tidak Dibutuhkan untuk Analisis Lebih Lanjut

In [118]:
# Menghilangkan (drop) kolom yang tidak dibutuhkan untuk analisis lebih lanjut pada dataset books
df_books = df_books.drop(['Image-URL-S', 'Image-URL-M', 'Image-URL-L'], axis=1)
df_books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


Pengecekan Tipe Data

In [119]:
# Tipe data books
df_books.dtypes

ISBN                   object
Book-Title             object
Book-Author            object
Year-Of-Publication    object
Publisher              object
dtype: object

Tipe data pada masing-masing kolom dataset ini telah sesuai.

In [120]:
# Tipe data users
df_users.dtypes

User-ID       int64
Location     object
Age         float64
dtype: object

Tipe data pada masing-masing kolom dataset ini telah sesuai.

In [121]:
# Tipe data ratings
df_ratings.dtypes

User-ID         int64
ISBN           object
Book-Rating     int64
dtype: object

Tipe data pada masing-masing kolom dataset ini telah sesuai.

Pengecekan Jumlah dan Rasio *Missing Value*

In [122]:
# Mengecek berapa jumlah null value pada masing masing kolom dataset books. 
jumlah_missing_books = pd.isnull(df_books).sum()
jumlah_missing_books

ISBN                   0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
dtype: int64

*   Jumlah *missing value* pada kolom Book-Author adalah 1
*   Jumlah *missing value* pada kolom Publisher adalah 2



In [123]:
# menghitung ratio missing value pada metrics books
rasio_missing_value_books = jumlah_missing_books.div(len(df_books))
rasio_missing_value_books

ISBN                   0.000000
Book-Title             0.000000
Book-Author            0.000004
Year-Of-Publication    0.000000
Publisher              0.000007
dtype: float64

Karena kolom Book-Author dan Publisher memiliki jumlah *missing value* dibawah 60% sehingga kedua kolom lebih baik tidak dihilangkan *(drop)* dari tabel.

In [124]:
# Mengecek berapa jumlah null value pada masing masing kolom dataset users. 
jumlah_missing_users = pd.isnull(df_users).sum()
jumlah_missing_users

User-ID          0
Location         0
Age         110762
dtype: int64

Kolom Age memiliki jumlah *missing value* sebesar 110762.

In [125]:
# menghitung ratio missing value pada metrics users
rasio_missing_value_users = jumlah_missing_users.div(len(df_users))
rasio_missing_value_users

User-ID     0.000000
Location    0.000000
Age         0.397199
dtype: float64

Kolom Age memiliki persentase *missing value* sebesar 39,72%. Oleh karena kolom Age memiliki persentase *missing value* dibawah 60% sehingga kolom ini lebih baik tidak dihilangkan *(drop)* dari tabel.

In [126]:
# Mengecek berapa jumlah null value pada masing masing kolom dataset ratings. 
jumlah_missing_ratings = pd.isnull(df_ratings).sum()
jumlah_missing_ratings

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

Tidak terdapat *missing value* pada seluruh kolom dalam dataset ini.

In [127]:
# menghitung ratio missing value pada metrics users
rasio_missing_value_ratings = jumlah_missing_ratings.div(len(df_ratings))
rasio_missing_value_ratings

User-ID        0.0
ISBN           0.0
Book-Rating    0.0
dtype: float64

Persentase *missing value* pada seluruh kolom dalam dataset ini adalah 0%.

Pengecekan *Duplicate Data*

In [128]:
# Cek data duplicate dari books
df_books[df_books.duplicated(keep=False)]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher


Tidak terdapat *duplicate data* pada seluruh kolom dataset ini.

In [129]:
# Cek data duplicate dari users
df_users[df_users.duplicated(keep=False)]

,User-ID,Location,Age


Tidak terdapat *duplicate data* pada seluruh kolom dataset ini.

In [130]:
# Cek data duplicate dari ratings
df_ratings[df_ratings.duplicated(keep=False)]

,User-ID,ISBN,Book-Rating


Tidak terdapat *duplicate data* pada seluruh kolom dataset ini.

#### EDA



In [131]:
# Melihat banyaknya user yang telah memberikan lebih dari 200 rating.
df_ratings['User-ID'].value_counts()

11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: User-ID, Length: 105283, dtype: int64

Jadi, banyaknya *user* yang telah memberikan lebih dari 200 *rating* adalah 105.283

In [132]:
# Mengekstrak User-ID yang telah memberikan lebih dari 200 rating
x = df_ratings['User-ID'].value_counts() > 200
y = x[x].index  #User_ID
print(y.shape)
ratings = df_ratings[df_ratings['User-ID'].isin(y)]

(899,)


In [133]:
# Melakukan merge ratings dengan books
rating_with_books = ratings.merge(df_books, on='ISBN')
rating_with_books.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc


#### FEATURE ENGINEERING

Metode sistem rekomendasi yang digunakan dalam *capstone project* ini adalah *collaborative based filtering* dengan algoritma *K-Nearest Neighbors*. *Collaborative based filtering* dapat dianalisis melalui *Matrix Factorization* dengan membuat tabel *book pivot*.

In [134]:
# Membuat final rating
number_rating = rating_with_books.groupby('Book-Title')['Book-Rating'].count().reset_index()
number_rating.rename(columns= {'Book-Rating':'number_of_ratings'}, inplace=True)
final_rating = rating_with_books.merge(number_rating, on='Book-Title')
final_rating.shape
final_rating = final_rating[final_rating['number_of_ratings'] >= 50]

Pada tahapan ini penyusun *project* akan menghitung *rating* setiap buku sehingga, penyusun dapat mengelompokkan data berdasarkan judul dan diagregasi berdasarkan *rating*.

In [135]:
# Menghilangkan duplikat users yang melakukan ratings pada buku yang sama
final_rating.drop_duplicates(['User-ID','Book-Title'], inplace=True)

Pada tahapan ini, data duplikat harus dibuang karena, jika *user* yang sama telah memberikan *rating* pada buku yang sama beberapa kali maka, akan menimbulkan masalah pada analisis lebih lanjut.

In [136]:
# Membuat tabel pivot
book_pivot = final_rating.pivot_table(columns='User-ID', index='Book-Title', values="number_of_ratings")
book_pivot.fillna(0, inplace=True)

Tabel pivot berisi akan berisi: 
* Kolom akan menjadi User-ID
* Index akan menjadi Book-Title
* *Value*-nya adalah Ratings

User-ID yang belum memberi *Ratings* pada buku apa pun akan memiliki nilai sebagai N/A, yang terhitung sebagai nol.

#### MODELING

In [137]:
# Konversi tabel pivot ke sparse matrix
book_sparse = csr_matrix(book_pivot)

Pada tahapan ini penyusun *project* melakukan konversi tabel pivot ke *sparse matrix* kemudian, memasukkannya ke model.

In [138]:
# Melakukan pemodelan K-Nearest Neighbors 
model = NearestNeighbors(algorithm='brute')
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

Pada tahapan ini penyusun *project* melakukan pemodelan *K-Nearest Neighbors*.

In [139]:
# Melakukan prediksi rekomendasi lima buku berdasarkan index 173 dengan Algoritma K-Nearest Neighbors
distances, suggestions = model.kneighbors(book_pivot.iloc[173, :].values.reshape(1, -1))

# Print judul bukunya
for i in range(len(suggestions)):
  print(book_pivot.index[suggestions[i]])

Index(['Dying for Chocolate (Culinary Mysteries (Paperback))',
       'Catering to Nobody', 'Tough Cookie', 'Prime Cut',
       'The Grilling Season'],
      dtype='object', name='Book-Title')


* Berdasarkan algoritma *K-Nearest Neighbors*, kelima indeks buku yang telah 
diprediksi adalah saling berkaitan.
* Kesamaan topik dari kelima buku pada *output* diatas adalah mengenai masak-memasak.
* Sehingga apabila terdapat penggemar/pembaca salah satu buku dari *output* diatas maka, bisa direkomendasikan untuk membaca empat buku lainnya dari *output* diatas.

In [140]:
# Melakukan prediksi rekomendasi lima buku berdasarkan index 237 dengan Algoritma K-Nearest Neighbors
distances, suggestions = model.kneighbors(book_pivot.iloc[237, :].values.reshape(1, -1))

# Print judul bukunya
for i in range(len(suggestions)):
  print(book_pivot.index[suggestions[i]])

Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Order of the Phoenix (Book 5)',
       'Harry Potter and the Sorcerer's Stone (Book 1)'],
      dtype='object', name='Book-Title')


* Berdasarkan algoritma *K-Nearest Neighbors*, kelima indeks buku yang telah 
diprediksi adalah saling berkaitan.
* Kesamaan topik dari kelima buku pada *output* diatas adalah mengenai Harry Potter.
* Sehingga apabila terdapat penggemar/pembaca salah satu buku dari *output* diatas maka, bisa direkomendasikan untuk membaca empat buku lainnya dari *output* diatas.

#### EVALUATION

In [141]:
# Mengakses judul buku yang berada di index 173
book_pivot.iloc[173, :]

User-ID
254       0.0
2276      0.0
2766      0.0
2977      0.0
3363      0.0
         ... 
275970    0.0
277427    0.0
277478    0.0
277639    0.0
278418    0.0
Name: Dying for Chocolate (Culinary Mysteries (Paperback)), Length: 888, dtype: float64

In [142]:
# Mengakses judul buku yang berada di index 237
book_pivot.iloc[237, :]

User-ID
254       183.0
2276        0.0
2766        0.0
2977        0.0
3363        0.0
          ...  
275970    183.0
277427    183.0
277478      0.0
277639      0.0
278418      0.0
Name: Harry Potter and the Chamber of Secrets (Book 2), Length: 888, dtype: float64

Menurut Tewari, Kumar dan Barman (2014) sistem rekomendasi banyak digunakan untuk merekomendasikan produk kepada *end-user* yang paling tepat. Situs web penjualan buku *online* saat ini bersaing satu sama lain dengan berbagai cara. Sistem rekomendasi adalah salah satu alat yang dapat mendukung situs web penjualan buku *online* lebih kuat untuk meningkatkan keuntungan dan mempertahankan pembeli.

Metode sistem rekomendasi yang digunakan dalam *capstone project* ini adalah *collaborative based filtering* dengan algoritma *K-Nearest Neighbors*. Lalu, menurut Singh, Sabharwal dan Gabrani (2022) *collaborative filtering* merekomendasikan item berdasarkan prinsip kesamaan *(similarity)*. Efektivitas *similarity*-nya sangat tergantung pada kinerja algoritma *clustering*. 

Pada *capstone project* ini telah terbukti bahwa algoritma *K-Nearest Neighbors* sangat efektif dalam merekomendasikan kepada penggemar/pembaca terkait, empat judul buku yang satu topik dengan buku pada index 173 (Dying for Chocolate (Culinary Mysteries (Paperback)) dan index 237 (Harry Potter and the Chamber of Secrets (Book 2)). Sehingga sistem rekomendasi ini dapat membuat website X lebih kuat untuk meningkatkan keuntungan dan mempertahankan para penggemar/pembacanya.

Empat buku yang memiliki kesamaan topik (masak-memasak) dengan buku pada index 173 (Dying for Chocolate (Culinary Mysteries (Paperback)) adalah:
*   Catering to Nobody
*   Tough Cookie
*   Prime Cut
*   The Grilling Season


Empat buku yang memiliki kesamaan topik (Harry Potter) dengan buku pada index 237 (Harry Potter and the Chamber of Secrets (Book 2)) adalah:
*   Harry Potter and the Prisoner of Azkaban (Book 3)
*   Harry Potter and the Goblet of Fire (Book 4)
*   Harry Potter and the Order of the Phoenix (Book 5)
*   Harry Potter and the Sorcerer's Stone (Book 1)


### KESIMPULAN/SARAN

Dari *capstone project* ini dapat disimpulkan bahwa, sistem pemberian rekomendasi sangat efektif dalam:
1. Memberikan rekomendasi kepada penggemar/pembaca mengenai, empat buku yang memiliki kesamaan topik (masak-memasak) dengan buku pada index 173 (Dying for Chocolate (Culinary Mysteries (Paperback)).
2. Memberikan rekomendasi kepada penggemar/pembaca mengenai, empat buku yang memiliki kesamaan topik (Harry Potter) dengan buku pada index 237 (Harry Potter and the Chamber of Secrets (Book 2)).

Adapun saran yang dapat diberikan kepada manajemen Website X dari *capstone project* ini adalah:
1. Melakukan intensifikasi sistem seperti pencarian topik buku lainnya dan pengembangan algoritma baru.
2. Melakukan ekstensifikasi sistem dengan penerapan algoritma *K-Nearest Neighbor* untuk pengembangan *Adaptive E-Learning Systems* di Website X, seperti pada Agarwal, Mishra dan Kolekar (2022).

### REFERENSI

Agarwal, A., Mishra, D. S. dan Kolekar, S. V. (2022) “Knowledge-based recommendation system using semantic web rules based on Learning styles for MOOCs,” *Cogent Engineering*. Cogent, 9(1), hal. 1–24. doi: 10.1080/23311916.2021.2022568.

Singh, V. K., Sabharwal, S. dan Gabrani, G. (2022) “A novel collaborative filtering based recommendation system using exponential grasshopper algorithm,” *Evolutionary Intelligence*. doi: 10.1007/s12065-021-00687-7.

Tewari, A. S., Kumar, A. dan Barman, A. G. (2014) “Book recommendation system based on combine features of content based filtering, collaborative filtering and association rule mining,” in *2014 IEEE International Advance Computing Conference (IACC)*. Gurgaon: IEEE, hal. 500–503. doi: 10.1109/IAdCC.2014.6779375.

